# ResNet50

Instalando requisitos

In [1]:
%pip install -r ./requirements.txt -q
%pip install pandas -q
%pip install scikit-learn -q
%pip install tensorflow -q
%pip install numpy -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Importando requisitos

In [2]:
from app.modelo.imagem_rgb import ImagemRGB
import sklearn.model_selection
import sklearn.preprocessing
import pandas
import tensorflow
import os
import numpy
import pickle
import random
import tensorflow.keras.preprocessing
import tensorflow.keras.applications.resnet50
import tensorflow.keras.models
import tensorflow.keras.layers

Definindo variáveis

In [3]:
image_directory = os.path.join(os.getcwd(), 'dataset_converted')
num_classes = 6

Abrindo base de dados

In [4]:
data_frame = pandas.read_csv("classifications.csv")

Adicionando coluna path

In [5]:
for index, row in data_frame.iterrows():
    data_frame.at[index, 'path'] = os.path.join(image_directory, row['bethesda_system'], "{}.png".format(row['cell_id']))

In [6]:
data_frame

,image_id,image_filename,image_doi,cell_id,bethesda_system,nucleus_x,nucleus_y,path
0,400,9ae8a4edde40219bad6303cebc672ee4.png,10.6084/m9.figshare.12230906,1,SCC,792,462,c:\Users\henrique\Trabalho_Pratico_-_Processam...
1,400,9ae8a4edde40219bad6303cebc672ee4.png,10.6084/m9.figshare.12230906,2,SCC,601,678,c:\Users\henrique\Trabalho_Pratico_-_Processam...
2,400,9ae8a4edde40219bad6303cebc672ee4.png,10.6084/m9.figshare.12230906,3,SCC,363,467,c:\Users\henrique\Trabalho_Pratico_-_Processam...
3,400,9ae8a4edde40219bad6303cebc672ee4.png,10.6084/m9.figshare.12230906,4,SCC,599,437,c:\Users\henrique\Trabalho_Pratico_-_Processam...
4,400,9ae8a4edde40219bad6303cebc672ee4.png,10.6084/m9.figshare.12230906,5,Negative for intraepithelial lesion,1186,450,c:\Users\henrique\Trabalho_Pratico_-_Processam...
...,...,...,...,...,...,...,...,...
11529,1,be340ee72689dfe3f8dc9c24de6127f4.png,10.6084/m9.figshare.12229511,11530,LSIL,618,407,c:\Users\henrique\Trabalho_Pratico_-_Processam...
11530,1,be340ee72689dfe3f8dc9c24de6127f4.png,10.6084/m9.figshare.12229511,11531,LSIL,607,374,c:\Users\henrique\Trabalho_Pratico_-_Processam...
11531,1,be340ee72689dfe3f8dc9c24de6127f4.png,10.6084/m9.figshare.12229511,11532,LSIL,707,251,c:\Users\henrique\Trabalho_Pratico_-_Processam...
11532,1,be340ee72689dfe3f8dc9c24de6127f4.png,10.6084/m9.figshare.12229511,11533,LSIL,579,246,c:\Users\henrique\Trabalho_Pratico_-_Processam...


Adicionando coluna random e ordenando por meio dela

In [7]:
data_frame['random'] = data_frame[data_frame.columns[0]].apply(lambda _: random.random())

In [8]:
data_frame.sort_values(
    by='random',
    inplace=True
)

Removendo imagens que não existem

In [9]:
for index, row in data_frame.iterrows():
    if not os.path.exists(row['path']):
        data_frame.drop(index, inplace=True)

In [10]:
data_frame

,image_id,image_filename,image_doi,cell_id,bethesda_system,nucleus_x,nucleus_y,path,random
5077,199,5ebaeeae59a5bc5ca0d79d99cb4dcac1.png,10.6084/m9.figshare.12230219,5078,Negative for intraepithelial lesion,1115,978,c:\Users\henrique\Trabalho_Pratico_-_Processam...,0.000337
5120,195,a7b7d4eb6367ad432a806b5662ff44e0.png,10.6084/m9.figshare.12230204,5121,Negative for intraepithelial lesion,1172,397,c:\Users\henrique\Trabalho_Pratico_-_Processam...,0.001105
11030,18,3659bf68aeea760b319cd0710e3651fb.png,10.6084/m9.figshare.12229577,11031,Negative for intraepithelial lesion,419,1000,c:\Users\henrique\Trabalho_Pratico_-_Processam...,0.001230
10825,25,f981c4aac622d7cce71a81305dd38ade.png,10.6084/m9.figshare.12229598,10826,Negative for intraepithelial lesion,1145,774,c:\Users\henrique\Trabalho_Pratico_-_Processam...,0.001232
5076,199,5ebaeeae59a5bc5ca0d79d99cb4dcac1.png,10.6084/m9.figshare.12230219,5077,Negative for intraepithelial lesion,813,876,c:\Users\henrique\Trabalho_Pratico_-_Processam...,0.001416
...,...,...,...,...,...,...,...,...,...
3997,249,bd7e1530680d11547db567150fa7c403.png,10.6084/m9.figshare.12230396,3998,Negative for intraepithelial lesion,364,424,c:\Users\henrique\Trabalho_Pratico_-_Processam...,0.999346
6537,135,8d62bc0267b58954ef9fa8e355d6ec22.png,10.6084/m9.figshare.12230006,6538,Negative for intraepithelial lesion,504,685,c:\Users\henrique\Trabalho_Pratico_-_Processam...,0.999892
3793,256,8932ac9bf855b72eef7390e7e486133e.png,10.6084/m9.figshare.12230423,3794,Negative for intraepithelial lesion,1017,703,c:\Users\henrique\Trabalho_Pratico_-_Processam...,0.999929
7337,114,624930d9bfc4219ba1bdd73ea3fed13b.png,10.6084/m9.figshare.12229913,7338,Negative for intraepithelial lesion,173,1000,c:\Users\henrique\Trabalho_Pratico_-_Processam...,0.999940


Separando o x

In [11]:
x = []
for index, row in data_frame.iterrows():
    x.append(row['path'])
x = pandas.array(x)

In [12]:
x

<StringArray>
[ 'c:\\Users\\henrique\\Trabalho_Pratico_-_Processamento_e_Analise_de_Imagens_-_2024-1\\develop_henrique_2024_06_01\\develop_henrique_2024_06_01\\dataset_converted\\Negative for intraepithelial lesion\\5078.png',
  'c:\\Users\\henrique\\Trabalho_Pratico_-_Processamento_e_Analise_de_Imagens_-_2024-1\\develop_henrique_2024_06_01\\develop_henrique_2024_06_01\\dataset_converted\\Negative for intraepithelial lesion\\5121.png',
 'c:\\Users\\henrique\\Trabalho_Pratico_-_Processamento_e_Analise_de_Imagens_-_2024-1\\develop_henrique_2024_06_01\\develop_henrique_2024_06_01\\dataset_converted\\Negative for intraepithelial lesion\\11031.png',
 'c:\\Users\\henrique\\Trabalho_Pratico_-_Processamento_e_Analise_de_Imagens_-_2024-1\\develop_henrique_2024_06_01\\develop_henrique_2024_06_01\\dataset_converted\\Negative for intraepithelial lesion\\10826.png',
  'c:\\Users\\henrique\\Trabalho_Pratico_-_Processamento_e_Analise_de_Imagens_-_2024-1\\develop_henrique_2024_06_01\\develop_henrique_

In [13]:
x.shape

(5581,)

Separando o y

In [14]:
y = data_frame.iloc[:, 4].values

In [15]:
y = sklearn.preprocessing.LabelEncoder().fit_transform(y)

Separando o conjunto de treino e teste

In [16]:
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y)

In [17]:
x_train.shape

(4185,)

In [18]:
y_train.shape

(4185,)

In [19]:
x_test.shape

(1396,)

In [20]:
y_test.shape

(1396,)

Exportando x_test, x_train, y_test, y_train para um arquivo externo

In [21]:
with open('data.pkl', mode='wb') as file:
    pickle.dump([x_train, x_test, y_train, y_test], file)

Preprocessando as imagens

In [22]:
def carregar_e_preprocessar_imagem(caminho_imagem):
    """
    Carrega e preprocessa uma imagem para ser utilizada em um modelo de aprendizado de máquina.
    """
    imagem = tensorflow.keras.preprocessing.image.load_img(caminho_imagem)
    #print(type(imagem))
    imagem_array = tensorflow.keras.preprocessing.image.img_to_array(imagem)
    #print(type(imagem_array))
    #print(imagem_array.shape)
    #imagem_array = numpy.expand_dims(imagem_array, axis=0)
    #print(type(imagem_array))
    #print(imagem_array.shape)
    return tensorflow.keras.applications.resnet50.preprocess_input(imagem_array)

In [23]:
carregar_e_preprocessar_imagem(x_train[0]).shape

(100, 100, 3)

In [24]:
type(carregar_e_preprocessar_imagem(x_train[0]).shape)

tuple

In [25]:
type(x_train[0])

str

# Aplicando o ResNet50

In [26]:
def predizer_classe_imagem(modelo, caminho_imagem):
    """
    Prediz a classe de uma imagem utilizando um modelo de aprendizado de máquina.
    """
    imagem_array = carregar_e_preprocessar_imagem(caminho_imagem)
    predicoes = modelo.predict(imagem_array)
    decoded_predictions = tensorflow.keras.applications.resnet50.decode_predictions(predicoes, top=3)[0]
    return decoded_predictions

Construindo o modelo

In [27]:
type(y_train)

numpy.ndarray

In [28]:
shape = (100, 100, 3)
modelo_basico = tensorflow.keras.applications.resnet50.ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=shape,
    )

In [29]:
x_ = tensorflow.keras.layers.GlobalAveragePooling2D()(modelo_basico.output) # modelo.output
x_ = tensorflow.keras.layers.Dense(len(numpy.unique(y)), activation='softmax')(x_)

In [30]:
modelo = tensorflow.keras.models.Model(inputs=modelo_basico.input, outputs=x_)

Compilando do modelo

In [31]:
modelo.compile(optimizer=tensorflow.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

Salvando o modelo

In [32]:
checkpoint = tensorflow.keras.callbacks.ModelCheckpoint('modelo_resnet_50.keras', monitor='val_accuracy', save_best_only=True)
parada_precoce = tensorflow.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, mode='min')

Treinando o modelo

In [33]:
type(y_train)

numpy.ndarray

In [34]:
x_train

<StringArray>
[                                'c:\\Users\\henrique\\Trabalho_Pratico_-_Processamento_e_Analise_de_Imagens_-_2024-1\\develop_henrique_2024_06_01\\develop_henrique_2024_06_01\\dataset_converted\\LSIL\\8784.png',
  'c:\\Users\\henrique\\Trabalho_Pratico_-_Processamento_e_Analise_de_Imagens_-_2024-1\\develop_henrique_2024_06_01\\develop_henrique_2024_06_01\\dataset_converted\\Negative for intraepithelial lesion\\4361.png',
  'c:\\Users\\henrique\\Trabalho_Pratico_-_Processamento_e_Analise_de_Imagens_-_2024-1\\develop_henrique_2024_06_01\\develop_henrique_2024_06_01\\dataset_converted\\Negative for intraepithelial lesion\\1097.png',
  'c:\\Users\\henrique\\Trabalho_Pratico_-_Processamento_e_Analise_de_Imagens_-_2024-1\\develop_henrique_2024_06_01\\develop_henrique_2024_06_01\\dataset_converted\\Negative for intraepithelial lesion\\7586.png',
  'c:\\Users\\henrique\\Trabalho_Pratico_-_Processamento_e_Analise_de_Imagens_-_2024-1\\develop_henrique_2024_06_01\\develop_henrique_

Preprocessando imagem

In [35]:
x_train_carregado = numpy.array([carregar_e_preprocessar_imagem(x_train[i]) for i in range(len(x_train))])
x_test_carregado = numpy.array([carregar_e_preprocessar_imagem(x_test[i]) for i in range(len(x_test))])

In [36]:
len(x_train)

4185

In [37]:
x_train_carregado.shape

(4185, 100, 100, 3)

In [38]:
x_train_carregado.shape

(4185, 100, 100, 3)

In [39]:
shapes = []
for i in range(len(x_train)):
    if x_train_carregado[i].shape not in shapes:
        shapes.append(x_train_carregado[i].shape)

In [40]:
shapes

[(100, 100, 3)]

In [41]:
y_train.shape

(4185,)

In [43]:
y_train

array([3, 4, 4, ..., 4, 4, 4])

In [44]:
modelo.fit(x_train_carregado, y_train, validation_data=(x_test_carregado, y_test), epochs=100, callbacks=[checkpoint, parada_precoce])

Epoch 1/100


ValueError: Arguments `target` and `output` must have the same rank (ndim). Received: target.shape=(None,), output.shape=(None, 6)

In [ ]:
#def train_image(modelo, image_path, classe):
    #array_imagem = carregar_e_preprocessar_imagem(image_path)
    #historico = modelo.fit(array_imagem, classe, epochs=10, validation_split=0.2, callbacks=[checkpoint, parada_precoce])
#

In [ ]:
type(x_train)

In [ ]:
#for i in range(len(x_train)):
#    train_image(modelo, x_train[i], y_train[i])

Exportando o modelo treinado

Exportando os pesos

Testando o modelo

Exportando as predições